# CS 484 :: Data Mining :: George Mason University :: Spring 2023


# Homework 3: Clustering&Association Rule Mining

- **100 points [9% of your final grade]**
- **Due Sunday, April 16 by 11:59pm**

- *Goals of this homework:* (1) implement your K-means model; and (2) implement the association rule mining process with the Apriori algorithm.

- *Submission instructions:* for this homework, you only need to submit to Blackboard. Please name your submission **FirstName_Lastname_hw3.ipynb**, so for example, my submission would be something like **Ziwei_Zhu_hw3.ipynb**. Your notebook should be fully executed so that we can see all outputs. 

## Part 1: Clustering (50 points)

In this part, you will implement your own K-means algorithm to conduct clustering on handwritten digit images. In this homework, we will still use the handwritten digit image dataset we have already used in previous homework. However, since clustering is unsupervised learning, which means we do not use the label information anymore. So, here, we will only use the testing data stored in the "test.txt" file.

First, let's load the data by excuting the following code:

In [40]:
import numpy as np

test = np.loadtxt("test.txt", delimiter=',')
test_features = test[:, 1:] # pixels for hand drawn image
test_labels = test[:, 0]    # actual number corresponding to features
print('array of testing feature matrix: shape ' + str(np.shape(test_features)))
print('array of testing label matrix: shape ' + str(np.shape(test_labels)))

array of testing feature matrix: shape (10000, 784)
array of testing label matrix: shape (10000,)


Now, it's time for you to implement your own K-means algorithm. First, please write your code to build your K-means model using the image data with **K = 10**, and **Euclidean distance**.

**Note: You should implement the algorithm by yourself. You are NOT allowed to use Machine Learning libraries like Sklearn**

**Note: you need to decide when to stop the model training process.**

In [55]:
# Write your code here
from tqdm import tqdm
K = 10 # number of clusters
FEATURE_SIZE = 784
num_features = len(test_features) # 10,000 samples
#-------------------------------------------------------------------------------
# select K points as initial centroids
# repeat:
#    form K clusters by assigning all points to the closest centroid
#    recompute the centroid of each cluster
# until: centroids dont change
#-------------------------------------------------------------------------------
centroid_indicies = np.random.randint(0,num_features,size=K,dtype=int)
centroids = []
for index in centroid_indicies:
    centroids.append(test_features[index])
centroids = np.asarray(centroids)
#print(centroids)
#-------------------------------------------------------------------------------
def cluster(points,centroids):
    current_centroids = centroids.copy()
    previous_centroids = np.zeros(centroids.shape,dtype=int)
    num_iterations = 0
    clusters = np.full(num_features,-1) #initialize to number that isnt a label
    # loop until centroids dont change
    while np.array_equal(current_centroids,previous_centroids) == False:
    #while True:
        previous_centroids = current_centroids.copy()
        num_iterations += 1
        
        # for every feature, calculate distance, find closest cluter and assign it
        for i in tqdm(range(num_features)):
            point = points[i]
            # distance formula from hw1 sol
            distances = np.sqrt(np.sum((point - current_centroids) ** 2, axis=1))
            closest = int(np.argmin(distances,axis=0))
            clusters[i] = closest
        # re-center centroid after updating cluster assignments
        for i in range(K):
            current_centroids[i] = np.mean(points[clusters == i],axis=0)

    print(f'Iterations: {num_iterations}')
    #print(current_centroids)
    return current_centroids,clusters
#-------------------------------------------------------------------------------
centroids_ret, clusters_ret = cluster(test_features,centroids)

num_correct = 0
for i in range(num_features):
    if clusters_ret[i] == test_labels[i]:
        num_correct += 1
print(num_correct)

100%|██████████| 10000/10000 [00:00<00:00, 92675.01it/s]

Iterations: 62
1684


Next, you need to calculate the square root of Sum of Squared Error (SSE) of each cluster generated by your K-means algorithm. Then, print out the averaged SSE of your algorithm.

In [56]:
# Write your code here
# create K element array to hold summed differences
cluster_sums = np.zeros((K,FEATURE_SIZE))

for index in clusters_ret:
    
    index = int(index)
    point = test_features[index]
    centroid = centroids_ret[index]
    difference = ((point - centroid)**2)
    cluster_sums[index] += difference
    #print(difference)

for i in range(K):
    print(f'Cluster [{i}] SSE: {np.round(np.sum(cluster_sums[i]),3)}')
print(f'Average SSE: {np.round(np.average(cluster_sums),3)}')

Cluster [0] SSE: 4504671715.729
Cluster [1] SSE: 3327420505.6
Cluster [2] SSE: 1282729718.223
Cluster [3] SSE: 5426438848.931
Cluster [4] SSE: 3707375979.528
Cluster [5] SSE: 4186860536.742
Cluster [6] SSE: 3385982558.484
Cluster [7] SSE: 3975840933.32
Cluster [8] SSE: 2531176689.801
Cluster [9] SSE: 4169276654.149
Average SSE: 4655328.334


Then, please have a look on https://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_completeness_v_measure.html#sklearn.metrics.homogeneity_completeness_v_measure, and use this function to print out the homogeneity, completeness, and v-measure of your K-means model.

In [57]:
# Write your code here
import sklearn.metrics
H,C,V = sklearn.metrics.homogeneity_completeness_v_measure(test_labels,clusters_ret) #gt,pred,beta
print(f'Homogeneity: {np.round(H,3)}')
print(f'Completeness: {np.round(C,3)}')
print(f'V-Measure: {np.round(V,3)}')

Homogeneity: 0.522
Completeness: 0.537
V-Measure: 0.529


## Part 2: Association Rule Mining (50 points)

In this part, you are going to examine movies using our understanding of association rules. For this part, you need to implement the apriori algorithm, and apply it to a movie rating dataset to find association rules of user-rate-movie behaviors. First, run the next cell to load the dataset we are going to use.

In [58]:
user_movie_data = np.loadtxt("movie_rated.txt", delimiter=',')
print('array of user-movie matrix: shape ' + str(np.shape(user_movie_data)))

array of user-movie matrix: shape (11743, 2)


In this dataset, there are two columns: the first column is the integer ids of users, and the second column is the integer ids of movies. Each row denotes that the user of given user id rated the movie of the given movie id. We are going to treat each user as a transaction, so you will need to collect all the movies that have been rated by a single user as a transaction. 

Now, you need to implement the apriori algorithm and apply it to this dataset to find association rules of user rating behaviors with **minimum support of 0.2** and **minimum confidence of 0.8**. We know there are many existing implementations of apriori online (check github for some good starting points). You are welcome to read existing codebases and let that inform your approach. 

**Note: Do not copy-paste any existing code.**

**Note: We want your code to have sufficient comments to explain your steps, to show us that you really know what you are doing.**

**Note: You should add print statements to print out the intermediate steps of your method -- e.g., the size of the candidate set at each step of the method, the size of the filtered set, and any other important information you think will highlight the method.**

In [59]:
# Write your code
from itertools import combinations
#include all the helpful print statements
# when you run this block, we want to see all of your intermediate steps
# you can save the rules you discover for printing in the following cells (this will help us grade by keeping these separate)
K = 2 # from lecture slides
# support - % of transactions that contain an interest
MIN_SUPPORT = 0.2
# confidence - how often items in Y appear in transactions that contain X
MIN_CONFIDENCE = 0.8

#MAX_ITERATIONS = 400 #TODO: change this

# combine all movies seen by one userID into a transaction

# split dataset into users and movies
total_users, total_movies = np.split(user_movie_data,2,axis=1)
# get unique elements from each array
unique_movies = np.unique(total_movies)
unique_users = np.unique(total_users)

num_unique_users = len(unique_users)
num_unique_movies = len(unique_users)
num_total_users = len(total_users)
num_total_movies = len(total_movies)

print(f'number of total users: {num_total_users}')
print(f'number of total movies: {num_total_movies}')
print(f'number of unique users: {num_unique_users}')
print(f'number of unique movies: {num_unique_movies}')


number of total users: 11743
number of total movies: 11743
number of unique users: 494
number of unique movies: 494


In [60]:
# create transactions from matrix data
# add to dictionary
# idk this seemed like a good way to store the transactions based on userID
def generate_transactions(dataset):
    transactions = {}
    num_entries,width = dataset.shape
    # for every user, see if theyre in transactions, if not, add
    for i in tqdm(range(num_entries)): # looping this way to be able to use TQDM for progress
        row = dataset[i]
        user = row[0]
        movie = row[1]
        # add every unique user to dict
        if user not in transactions.keys():
            transactions[user] = []
        if movie not in transactions[user]:
            transactions[user].append(movie)
    return transactions
#print(transactions)
# get transactions
transactions = generate_transactions(user_movie_data)
# convert transactions into array
transaction_list = list(transactions.values())


100%|██████████| 11743/11743 [00:00<00:00, 1542263.02it/s]


In [61]:
# make sure number of keys in transactions is the same as the number of unique users
print(len(transaction_list)==num_unique_users)
num_transactions = num_unique_users
print(f'number of transactions: {len(transaction_list)}')

True
number of transactions: 494


In [62]:
# [x] -> [Y] format
K = 2
#-------------------------------------------------------------------------------
# method for computing support for list of movies
def calculate_support(_dict,movie):
    return _dict.get(movie)/(len(transaction_list))
#-------------------------------------------------------------------------------
# method for computing confidence for a list of movies
# = support(X)/support(Y)
#TODO: implement confidence
def confidence(X,Y,_dict):
    return calculate_support(_dict,X.union(Y))/calculate_support(_dict,X)
#-------------------------------------------------------------------------------
def generate_initial_frequent_set(transactions):
    movie_count = {}
    for i in tqdm(range(len(transactions))):
        transaction = transactions[i]
        # inc count for every movie, create K,V pair if DNE
        for movie in transaction:
            if movie not in movie_count.keys():
                movie_count[movie] = 0
            movie_count[movie] += 1
    # create 1 element frequent set
    frequent_set = []
    for movie in movie_count.keys():
        support = calculate_support(movie_count,movie)
        if support >= MIN_SUPPORT:
            frequent_set.append(movie)
    return frequent_set
#-------------------------------------------------------------------------------
frequent_set = generate_initial_frequent_set(transaction_list)
len_frequent_itemset_1 = len(frequent_set)
print(f'size of 1 elemenent frequent set: {len_frequent_itemset_1}')
#-------------------------------------------------------------------------------
def candidate_sets_combinations(frequent_set,K):
    stop = False
    # loop until no new candidates are added to frequent set
    while stop == False:
        print(f'frequent_set: {frequent_set}')
        print(f'K: {K}')
        print(f'Generating {K-1} element combos')
        num_added = 0
        candidate_count = {}
        # generate K length combinations of frequent set
        candidate_set = combinations(frequent_set,K)
        candidate_set = list(candidate_set) # convert to list of tuples
        # count occurances of candidates for support
        for transaction in transaction_list:
            for candidate in candidate_set:
                #print(f'candidate: {candidate}')
                #print(f'transaction: {transaction}')
                # if candidate is in transaction, inc count
                if set(candidate).issubset(set(transaction)):
                    #print('in here!')
                    if candidate not in candidate_count.keys():
                        candidate_count[candidate] = 0
                    #if candidate in candidate_count.keys():
                    candidate_count[candidate] += 1

        # add supporting candidates to frequent set
        print(f'Number of candidate combos: {len(candidate_count.keys())}')
        for candidate in candidate_count.keys():
            support = calculate_support(candidate_count,candidate)
            #print(support)
            if support >= MIN_SUPPORT:
                print(f'Added: {candidate}')
                num_added += 1
                frequent_set.append(candidate)
        K += 1
        if num_added == 0:
            stop = True
            print('Breaking!')
            break
    return frequent_set
#-------------------------------------------------------------------------------
# generate candidate sets
# OLD!!!!!
def generate_candidate_sets(frequent_set):
    return
#-------------------------------------------------------------------------------
# OLD
#frequent_set = generate_candidate_sets(frequent_set)
#print(f'len frequent set: {len(frequent_set)}')
#print(frequent_set)

frequent_set = candidate_sets_combinations(frequent_set,K)
print(f'Length of frequent set: {len(frequent_set)}')
print(frequent_set)

100%|██████████| 494/494 [00:00<00:00, 276892.45it/s]


size of 1 elemenent frequent set: 21
frequent_set: [480.0, 1221.0, 1270.0, 1265.0, 1197.0, 260.0, 1961.0, 527.0, 2028.0, 2987.0, 858.0, 1584.0, 2710.0, 1094.0, 1252.0, 587.0, 1387.0, 3255.0, 2706.0, 377.0, 1127.0]
K: 2
Generating 1 element combos
Number of candidate combos: 210
Added: (1270.0, 1265.0)
Added: (1270.0, 1197.0)
Added: (1270.0, 260.0)
Added: (1270.0, 527.0)
Added: (1270.0, 2028.0)
Added: (1265.0, 1197.0)
Added: (1265.0, 260.0)
Added: (1265.0, 527.0)
Added: (1265.0, 2028.0)
Added: (1197.0, 260.0)
Added: (1197.0, 527.0)
Added: (1197.0, 2028.0)
Added: (260.0, 527.0)
Added: (260.0, 2028.0)
Added: (527.0, 2028.0)
Added: (480.0, 2028.0)
Added: (480.0, 1270.0)
Added: (480.0, 1265.0)
Added: (480.0, 1197.0)
Added: (480.0, 260.0)
Added: (480.0, 527.0)
Added: (480.0, 858.0)
Added: (1270.0, 2987.0)
Added: (1270.0, 858.0)
Added: (1197.0, 2987.0)
Added: (1197.0, 858.0)
Added: (260.0, 2987.0)
Added: (260.0, 858.0)
Added: (527.0, 858.0)
Added: (1221.0, 260.0)
Added: (1221.0, 858.0)
Added:

In [63]:
# generate rules from frequent set
frequent_set_copy = frequent_set.copy()
def get_rules(frequent_set):
    association_rules = []
    for _set in frequent_set:
        combos = []
        print(_set)
        # if single element in set
        if isinstance(_set,np.floating):
            combos.append(_set)
        else:
            # if two element set
            if len(_set) == 2:
                combos.append([_set[0],_set[1]])
            # if > 2 element set
            else:
                for i in range(len(_set)):
                    for j in range(i+1,len(_set)):
                        combos.append([_set[i],_set[j]])
    # for each rule generated, calculate confidence and if >= MIN_CONFIDNECE, add to assoc_rules
    
    print(f'combos: {combos}')
get_rules(frequent_set_copy)
print

480.0
1221.0
1270.0
1265.0
1197.0
260.0
1961.0
527.0
2028.0
2987.0
858.0
1584.0
2710.0
1094.0
1252.0
587.0
1387.0
3255.0
2706.0
377.0
1127.0
(1270.0, 1265.0)
(1270.0, 1197.0)
(1270.0, 260.0)
(1270.0, 527.0)
(1270.0, 2028.0)
(1265.0, 1197.0)
(1265.0, 260.0)
(1265.0, 527.0)
(1265.0, 2028.0)
(1197.0, 260.0)
(1197.0, 527.0)
(1197.0, 2028.0)
(260.0, 527.0)
(260.0, 2028.0)
(527.0, 2028.0)
(480.0, 2028.0)
(480.0, 1270.0)
(480.0, 1265.0)
(480.0, 1197.0)
(480.0, 260.0)
(480.0, 527.0)
(480.0, 858.0)
(1270.0, 2987.0)
(1270.0, 858.0)
(1197.0, 2987.0)
(1197.0, 858.0)
(260.0, 2987.0)
(260.0, 858.0)
(527.0, 858.0)
(1221.0, 260.0)
(1221.0, 858.0)
(2028.0, 858.0)
(480.0, 377.0)
(260.0, 1387.0)
(260.0, 377.0)
(260.0, 1127.0)
(1270.0, 1265.0, 1197.0)
(1270.0, 1265.0, 260.0)
(1270.0, 1197.0, 260.0)
(1270.0, 260.0, 527.0)
(1270.0, 260.0, 2028.0)
(1265.0, 1197.0, 260.0)
(1197.0, 260.0, 2028.0)
(260.0, 527.0, 2028.0)
(480.0, 1270.0, 260.0)
(480.0, 1197.0, 260.0)
(1221.0, 260.0, 858.0)
(480.0, 260.0, 2028.0)


<function print>

Finally, print your final association rules in the following format:

**movie_name_1, movie_name_2, ... --> movie_name_k**

where the movie names can be fetched by joining the movieId with the file 'movies.csv'. For example, one rule that you should find is:

**Jurassic Park (1993), Back to the Future (1985) --> Star Wars: Episode IV - A New Hope (1977)**

**Hint: You may need to use the Pandas library to load and process the movies.csv file, such as use pandas.read_csv() to load the data. https://pandas.pydata.org/pandas-docs/dev/user_guide/10min.html is a good place to learn the basics about Pandas.**

In [50]:
import pandas as pd
# Write your code to print out the rules
movie_names = pd.read_csv('movies.csv')
movie_names.head()

frequent_set_copy = frequent_set.copy()

#print(movie_names.iloc[4][1])
# loop over every rule in rules
for _ in _:
    # loop over every element in each rule
    for _ in _:
        movie = movie_names.iloc[int(movie)][1]
print(frequent_set_copy)

TypeError: 'builtin_function_or_method' object is not iterable